# Möbius Plateau Explorer v5 🔍

**Purpose**
- Load your fine‑scan table *(bits, k, φ, λ, rel_err)*
- Auto‑bucket rows into plateaux (tunable ε slider)
- Apply quick ops (÷9, ×φ, ÷α, δ, Feigenbaum goodies…)
- Stack ops (`÷9 ×φ /α_half`, &c.)
- Mini charts update live so you can eyeball structure.
- One‑click export of **whatever** view you end on.

> φ ≈ 0.00394344 ± 5×10⁻⁷ α⁻¹ ≈ 137.035999084 δ ≈ 4.669201609 α_feig ≈ 2.502907875


In [1]:
import pandas as pd, numpy as np, ipywidgets as W

# 🔵🔵 PASTE / UPDATE YOUR RAW ROWS BELOW 🔵🔵
raw_rows = [
    (64,3.6,0.003942,0.0,0.41175258759066782690422624774513012754204638),
    (64,3.8,0.003942,0.0,0.41175258759066782690422624774513012754204638),
    (64,4.0,0.003942,0.0,0.41175258759066782690422624774513012754204638),
    (64,4.2,0.003942,0.0,0.41175258759066782690422624774513012754204638),
    (128,3.6,0.003942,0.0,0.41175258759066782705725515126342766544650360),
    (128,3.8,0.003942,0.0,0.41175258759066782705725515126342766544650360),
    (128,4.0,0.003942,0.0,0.41175258759066782705725515126342766544650360),
    (128,4.2,0.003942,0.0,0.41175258759066782705725515126342766544650360),
]

df = pd.DataFrame(raw_rows, columns=['bits','k','phi','lam','rel_err'])
display(df.head())

,bits,k,phi,lam,rel_err
0,64,3.6,0.003942,0.0,0.411753
1,64,3.8,0.003942,0.0,0.411753
2,64,4.0,0.003942,0.0,0.411753
3,64,4.2,0.003942,0.0,0.411753
4,128,3.6,0.003942,0.0,0.411753


In [2]:
# ==== Plateau detector with live ε slider ====
eps_slider = W.FloatLogSlider(value=1e-5, base=10, min=-10, max=-1, step=0.25,
                             description='ε (rel_err)', continuous_update=False)

def tag(df_in, eps):
    out = df_in.copy()
    base = out['rel_err'].min()
    out['plateau'] = ((out['rel_err'] - base)/eps).astype(int)
    return out

out_tbl = W.Output()
def refresh(*_):
    with out_tbl:
        out_tbl.clear_output()
        tagged = tag(df, eps_slider.value)
        display(tagged.groupby('plateau')['rel_err'].agg(['min','max','count']))
        display(tagged)

eps_slider.observe(refresh, 'value'); refresh()
display(eps_slider, out_tbl)

FloatLogSlider(value=1e-05, continuous_update=False, description='ε (rel_err)', max=-1.0, min=-10.0, step=0.25…

Output()

In [3]:
# ==== Quick‑op helper ====
alpha_inv = 137.035999084
phi_c = 0.00394344 + 5e-7
phi2 = phi_c**2
delta = 4.669201609; delta2 = delta**2; delta_half = delta/2
alpha_f = 2.502907875; alpha_f2 = alpha_f**2; alpha_half = alpha_inv/2

def eval_token(tok):
    consts = {
        '9':9, '1/9':1/9, 'φ':phi_c, 'φ2':phi2,
        'α':alpha_inv, 'α2':alpha_inv**2, 'α/2':alpha_half,
        'δ':delta, 'δ2':delta2, 'δ/2':delta_half,
        'αf':alpha_f, 'αf2':alpha_f2,
    }
    return consts.get(tok)

def apply_ops(series, op_str):
    tokens = op_str.split()
    out = series.copy()
    for t in tokens:
        if t.startswith('×') or t.startswith('*'):
            val = eval_token(t[1:]); out *= val
        elif t.startswith('÷') or t.startswith('/'):
            val = eval_token(t[1:]); out /= val
        elif t.startswith('+'):
            val = eval_token(t[1:]); out += val
        elif t.startswith('-'):
            val = eval_token(t[1:]); out -= val
    return out

op_box = W.Text(value='', placeholder='e.g. /9 ×φ ×δ', description='Ops')
apply_btn = W.Button(description='Apply', button_style='success')
out_ops = W.Output()

def run_ops(_):
    with out_ops:
        out_ops.clear_output()
        tagged = tag(df, eps_slider.value)
        view = tagged.copy()
        if op_box.value.strip():
            view['rel_err'] = apply_ops(view['rel_err'], op_box.value)
        display(view)
apply_btn.on_click(run_ops)
display(W.HBox([op_box, apply_btn]), out_ops)

Output()

In [4]:
# ==== Export current view ====
file = 'plateaus_v5.xlsx'
df.to_excel(file, index=False)
print('Saved to', file)

Saved to plateaus_v5.xlsx
